In [1]:
from g4f.client import Client

### Prompt generation engine


In [16]:
import json

def generate_prompt(json_input):
    """
    Generates a prompt from a given JSON input.

    Args:
        json_input (str): JSON string containing 'task', 'example', and 'output'.

    Returns:
        str: Generated prompt.
    """
    
    # Load the JSON input
    data = json.loads(json_input)
    
    # Extracting the fields from JSON
    task = data.get('task', '')
    example = data.get('example', '')
    output = data.get('output', '')

    system_prompt = '''
        You are provided with a very important task. Take as many iterations as you need to complete the task. \
        You must follow strict guidelines to ensure the task is completed correctly. 
    '''

    # Constructing the prompt
    prompt = '''
       
        The guidelines are as follows: 
        ``` 
            1. **Follow the task description:** Always prioritize completing the assigned task as outlined in the initial prompt. 
            2. **Use the example input/output as a guide:**  Base your responses on the provided format and style. 
            3. **Incorporate user-provided examples:**  Learn and adapt from any additional examples the user provides to further refine your understanding of the desired output. 
            4. **Maintain factual accuracy:**  Strive to generate outputs that are consistent with established knowledge and avoid making claims that cannot be substantiated. 
            5. **Prioritize clarity and coherence:**  Ensure your outputs are easy to understand and logically structured. 
            6. **Be creative within the defined parameters:**  While adhering to the task and examples, feel free to use your knowledge to generate informative and engaging content.
            7. **Return only the prompt:**  Your final response should only include the prompt generated based on the user input.
            8. **The language must be English:**  The prompt should be generated in English.
        ```\

        The task: ```Generate a prompt that would take as input 3 variables: 'user_task', 'user_example' and 'user_output' (optional). In case the output is not provided, you decide what output fits the best. The prompt must be very descriptive. ```\
        The input example: ``` 
            {
                "task": "HTML to JSON conversion",
                "example": "<h1>This is a heading</h1>\
                            <p>This is a paragraph with some text.</p>\
                            <ul>\
                            <li>Item 1</li>\
                            <li>Item 2</li>\
                            </ul>",
                "output": "{
                                "type": "document",
                                "children": [
                                    { "type": "heading", "text": "This is a heading" },
                                    { "type": "paragraph", "text": "This is a paragraph with some text." },
                                    { "type": "list", "items": ["Item 1", "Item 2"] }
                                ]
                            }"
            }
        ```\
        The output example: ``` 
            **1. Introduction:**\
            I am providing you with an HTML string. Please convert it into a well-structured JSON representation that captures the essential elements and their relationships. Here's an example to illustrate the desired format:\
            
            **2. Example:**\
            
            HTML:\
            ```HTML
            <h1>This is a heading</h1>\
            <p>This is a paragraph with some text.</p>\
            <ul>\
            <li>Item 1</li>\
            <li>Item 2</li>\
            </ul>
            ```\
            
            JSON:\
            ```JSON
            {
                "type": "document",
                "children": [
                    { "type": "heading", "text": "This is a heading" },
                    { "type": "paragraph", "text": "This is a paragraph with some text." },
                    { "type": "list", "items": ["Item 1", "Item 2"] }
                ]
            }
            ```

            **3. Processing Instructions:**\
            1. Parse the HTML string: Analyze the HTML structure and identify elements like headings, paragraphs, lists, etc. and their content.
            2. Create a JSON object: Construct a JSON object with a "type" property representing the overall document type (e.g., "document").
            3. Define child elements: Within the main object, create a "children" array to hold information about child elements.
            4. Map elements to JSON structure: For each element in the HTML, create a corresponding object within the "children" array.
            5. Include a "type" property specifying the element type (e.g., "heading", "paragraph", "list").
            6. Add a "text" property for elements containing text content (e.g., headings, paragraphs).
            7. For lists, include an "items" array containing the list items.
            8. Handle complex elements: While this example covers basic elements, be prepared to handle more complex structures like tables, forms, and nested elements using appropriate JSON representations.
        ```
    '''

    prompt += f'''
        The user task: ```{task}```\
        The user example: ```{example}```\
        The user output: ```{output}```\
    '''

    client_local = Client()
    response = client_local.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": system_prompt + prompt
            }
        ]
    )
    return response.choices[0].message.content
    

### Example 1 with output


In [12]:
task_dict = {
    "task": "Transform the unstructured data into structured data.",
    "example": "My neighbor Jhon said that he would be coming to the party. I said that I would be coming too. Then I met my other neighbor Andrew and said that I dont want to see Jhon at the party. Andrew said that he would be coming too.",
    "output": "<Jhon> I would be coming to the party. \n <Me> I would be coming too. \n[Met Andrew.] \n <Me> I dont want to see Jhon at the party. \n <Andrew> I would be coming too."
}

json_input = json.dumps(task_dict)

prompt = generate_prompt(json_input)
print(prompt)


**1. Introduction:**
I am providing you with an unstructured data string. Please convert it into a well-structured representation that captures the essential elements and their relationships. Here's an example to illustrate the desired format:

**2. Example:**
Unstructured Data:
"My neighbor Jhon said that he would be coming to the party. I said that I would be coming too. Then I met my other neighbor Andrew and said that I dont want to see Jhon at the party. Andrew said that he would be coming too."

Structured Data:
```
[
    {
        "speaker": "Jhon",
        "message": "I would be coming to the party."
    },
    {
        "speaker": "Me",
        "message": "I would be coming too."
    },
    {
        "event": "Met Andrew."
    },
    {
        "speaker": "Me",
        "message": "I dont want to see Jhon at the party."
    },
    {
        "speaker": "Andrew",
        "message": "I would be coming too."
    }
]
```

**3. Processing Instructions:**
1. Parse the unstructured data

### Example 2 without output


In [11]:
task_dict = {
    "task": "Answer the question",
    "example": "What is the capital of France?",
    "output": ""
}

json_input = json.dumps(task_dict)

prompt = generate_prompt(json_input)
print(prompt)


**1. Introduction:**
I am providing you with a question. Please answer it accurately. Here's an example to illustrate the desired format:

**2. Example:**
Question: What is the capital of France?
Answer: Paris

**3. Processing Instructions:**
1. Read the question: Understand the context and content of the question.
2. Retrieve relevant information: Access knowledge databases or perform necessary calculations to find the correct answer.
3. Provide a concise response: Offer a clear and direct answer to the question.
4. Ensure accuracy: Verify the answer against reliable sources to ensure correctness.
5. Handle follow-up questions: Be prepared to answer related questions or provide additional information if needed.

Please provide the answer to the question: What is the capital of France?


In [14]:
from g4f.Provider import ChatgptLogin

new_task_examples = {
    "examples": ["What would be the output of the following code snippet?\n\n```python\nx = 5\ny = 10\nz = x + y\nprint(z)\n```\n\n",]
}


prompt_with_new_example = prompt + "\n\n" + "**User Examples:**\n"
for example in new_task_examples["examples"]:
    prompt_with_new_example += example

client = Client()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": prompt_with_new_example + "Return the answer in english."
        }
    ]
)
print(response.choices[0].message.content)

To convert the given unstructured data string into a well-structured representation, I will analyze the string to identify speakers, messages, and events. Then, I'll create a structured format to hold information about the conversation.

Here's the unstructured data string provided:
"What would be the output of the following code snippet? ```python x = 5 y = 10 z = x + y print(z)``` Return the answer in english."

And here is the structured data representation for it:

```json
[
    {
        "event": "Asked about the output of a code snippet."
    },
    {
        "code_snippet": [
            "x = 5",
            "y = 10",
            "z = x + y",
            "print(z)"
        ]
    },
    {
        "request": "Return the answer in English."
    }
]
```

Explanation:
- The initial part of the string is identified as an event where the user asks about the output of a code snippet.
- The code snippet itself is structured as an array of code lines.
- The final part is a request to retu

In [18]:
task = '''
    I run a travel blog and want to create a series of articles about hidden gems in Europe. I have a list of 20 lesser-known cities with basic info like country, population, and a few key attractions. I want to generate engaging blog posts that include a catchy title, an introduction that piques interest, a section on the city's history, a part about local cuisine, and a conclusion that encourages readers to visit. Each post should be around 800 words.
'''
example = '''
    Select a city from the list of 20 lesser-known cities.
'''

output = ''''''

task_dict = {
    "task": task,
    "example": example,
    "output": output
}

json_input = json.dumps(task_dict)

prompt = generate_prompt(json_input)
print(prompt)


**1. Introduction:**
I am providing you with a task to create engaging blog posts about hidden gems in Europe. Each post should be around 800 words and include specific sections to captivate the reader. Below is an example to guide you in generating the content.

**2. Example:**
Task:
```
I run a travel blog and want to create a series of articles about hidden gems in Europe. I have a list of 20 lesser-known cities with basic info like country, population, and a few key attractions. I want to generate engaging blog posts that include a catchy title, an introduction that piques interest, a section on the city's history, a part about local cuisine, and a conclusion that encourages readers to visit. Each post should be around 800 words.
```
Example City Information:
```
City: Ljubljana
Country: Slovenia
Population: 280,000
Key Attractions: Ljubljana Castle, Tivoli Park, Triple Bridge
```

**3. Processing Instructions:**
1. **Select a city:** Choose one city from the provided list of 20 le